# **Pneumonia Detection by CNN with Data Augmentation** 

2020/03/15
+ 本範例利用 TensorFlow 2.0 架構下的 `tf.keras` 套件，來建立 CNN 模型，透過胸腔 X-光片影像進行肺炎偵測。
+ 資料部份：
    - 將 "Chest_Xray" 影像資料匯入，並修改、統一其影像尺寸為 (224, 224, 3)。
    - 其中，資料匯入區分為 train、test 和 val 三個部分。

---------------------------
## CONTENT
1. [ APPROACH ](#approach)
2. [ Data Preprocessing ](#preprocessing)
    + [ Importing Training Datasets ](#TrainData)
    + [ Importing Validation Datasets ](#ValData)
    + [ Importing Test Datasets ](#TestData)
3. [ CNN Model with *tf.keras* ](#CNNModel)
    + [ Forward Propagation ](#Forwardpropagation)
    + [ Model Summary & Plotting the Model ](#ModelSummary)
- [ Start Training - from Coarse to Fine ](#StartTraining)
    + [ Setting Hyperparameters ](#SettingHyperparameters)
    + [ STAGE 1 - Coarse Training without Data Augmentation ](#Stage1)
        + [ Backpropagation for STAGE 1 ](#Backpropagation1)
    + [ STAGE 2 - Fine Training *with* Data Augmentation ](#Stage2)
        + [ Backpropagation for STAGE 2 ](#Backpropagation2)
- [ Saving the Entire Model with HDF5 Format ](#SavingEntireModel)
---------------------------

<a id='approach'></a>
## 1. APPROACH
> + 本範例採用 **From Coarse to Fine** 的調校過程 (Tuning Process)：
    - 首先，利用原始影像資料(raw images)進行 CNN Model 參數調校，並輸出結果 (參考 < STAGE 1 : Coarse Training without Data Augmentation > 部份)。
    - 之後，利用 "**資料擴增 (Data Augmentation)**" 技術，在程式執行階段 (runtime) 增加 Training 資料量，再次對 CNN Model 進行參數調校，並輸出結果 (參考 < STAGE 2 > 部份)。
> + 其**目的**是在 "**有效縮短運算時間，減少佔用記憶體空間，同時提升預測結果的準確度**"，達成整體效能(performance)的提升。
* > + 本範例中，**CNN Model** 採用 "Batch Normalization"、"Dropout"以及 "Learning-Rate Decay" 等技術。
> + 下列程式同時輸出 < STAGE 1 > 和 < STAGE 2 > 結果做為參考，並輸出、儲存整個預測模型。

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import seaborn as sns
%matplotlib inline
from sklearn.metrics import confusion_matrix

import cv2
import os
import glob

from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, SeparableConv2D
from tensorflow.keras.layers import GlobalMaxPooling2D, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

<a id='preprocessing'></a>
## 2. Data Preprocessing

In [ ]:
# Input data files are available in the "../input/" directory.
INPUT_PATH = "../input/pneumonia-detection/chest_xray"

# List the files in the input directory.
print(os.listdir(INPUT_PATH))

> ### [ File Directory ]
       +-- input 
             |-- pneumonia-detection
                  |-- chest_xray
                         |-- test
                               |-- NORMAL
                               |-- PNEUMONIA
                         |-- train
                               |-- NORMAL
                               |-- PNEUMONIA
                         |-- val
                               |-- NORMAL
                               |-- PNEUMONIA


### Training Datasets

In [ ]:
# list of all the training images
train_normal = Path(INPUT_PATH + '/train/NORMAL').glob('*.jpeg')
train_pneumonia = Path(INPUT_PATH + '/train/PNEUMONIA').glob('*.jpeg')

# ---------------------------------------------------------------
# Train data format in (img_path, label) 
# Labels for [ the normal cases = 0 ] & [the pneumonia cases = 1]
# ---------------------------------------------------------------
normal_data = [(image, 0) for image in train_normal]
pneumonia_data = [(image, 1) for image in train_pneumonia]

train_data = normal_data + pneumonia_data

# Get a pandas dataframe from the data we have in our list 
train_data = pd.DataFrame(train_data, columns=['image', 'label'])

# Checking the dataframe...
train_data.head()

In [ ]:
# Checking the dataframe...
train_data.tail()

In [ ]:
# Shuffle the data 
train_data = train_data.sample(frac=1., random_state=100).reset_index(drop=True)

# Checking the dataframe...
train_data.head(10)

In [ ]:
print(train_data)

In [ ]:
# Counts for both classes
count_result = train_data['label'].value_counts()
print('Total of Train Data : ', len(train_data), '  (0 : Normal; 1 : Pneumonia)')
print(count_result)

# Plot the results 
plt.figure(figsize=(8,5))
sns.countplot(x = 'label', data =  train_data)
plt.title('Number of classes', fontsize=16)
plt.xlabel('Class type', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(range(len(count_result.index)), 
           ['Normal : 0', 'Pneumonia : 1'], 
           fontsize=14)
plt.show()

In [ ]:
fig, ax = plt.subplots(3, 4, figsize=(20,15))
for i, axi in enumerate(ax.flat):
    image = imread(train_data.image[i])
    axi.imshow(image, cmap='bone')
    axi.set_title(('Normal' if train_data.label[i] == 0 else 'Pneumonia') 
                  + '  [size=' + str(image.shape) +']',
                  fontsize=14)
    axi.set(xticks=[], yticks=[])

In [ ]:
train_data.to_numpy().shape

### Import X-ray Image Datasets from /train, /val & /test

In [ ]:
# ----------------------------------------------------------------------
#  Loading X-ray Images datasets from file 3 directories, respectively. 
# ----------------------------------------------------------------------
def load_data(files_dir='/train'):
    # list of the paths of all the image files
    normal = Path(INPUT_PATH + files_dir + '/NORMAL').glob('*.jpeg')
    pneumonia = Path(INPUT_PATH + files_dir + '/PNEUMONIA').glob('*.jpeg')

    # --------------------------------------------------------------
    # Data-paths' format in (img_path, label) 
    # labels : for [ Normal cases = 0 ] & [ Pneumonia cases = 1 ]
    # --------------------------------------------------------------
    normal_data = [(image, 0) for image in normal]
    pneumonia_data = [(image, 1) for image in pneumonia]

    image_data = normal_data + pneumonia_data

    # Get a pandas dataframe for the data paths 
    image_data = pd.DataFrame(image_data, columns=['image', 'label'])
    
    # Shuffle the data 
    image_data = image_data.sample(frac=1., random_state=100).reset_index(drop=True)
    
    # Importing both image & label datasets...
    x_images, y_labels = ([data_input(image_data.iloc[i][:]) for i in range(len(image_data))], 
                         [image_data.iloc[i][1] for i in range(len(image_data))])

    # Convert the list into numpy arrays
    x_images = np.array(x_images)
    y_labels = np.array(y_labels)
    
    print("Total number of images: ", x_images.shape)
    print("Total number of labels: ", y_labels.shape)
    
    return x_images, y_labels

In [ ]:
# ---------------------------------------------------------
#  1. Resizing all the images to 224x224 with 3 channels.
#  2. Then, normalize the pixel values.  
# ---------------------------------------------------------
def data_input(dataset):
    # print(dataset.shape)
    for image_file in dataset:
        image = cv2.imread(str(image_file))
        image = cv2.resize(image, (224,224))
        if image.shape[2] == 1:
            # np.dstack(): Stack arrays in sequence depth-wise 
            #              (along third axis).
            # https://docs.scipy.org/doc/numpy/reference/generated/numpy.dstack.html
            image = np.dstack([image, image, image])
        
        # ----------------------------------------------------------
        # cv2.cvtColor(): The function converts an input image 
        #                 from one color space to another. 
        # [Ref.1]: "cvtColor - OpenCV Documentation"
        #     - https://docs.opencv.org/2.4/modules/imgproc/doc/miscellaneous_transformations.html
        # [Ref.2]: "Python计算机视觉编程- 第十章 OpenCV" 
        #     - https://yongyuan.name/pcvwithpython/chapter10.html
        # ----------------------------------------------------------
        x_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Normalization
        x_image = x_image.astype(np.float32)/255.
        return x_image

<a id="TrainData"></a>
+ ### Importing Training Datasets

In [ ]:
# Import train dataset...
x_train, y_train = load_data(files_dir='/train')

print(x_train.shape)
print(y_train.shape)

In [ ]:
x_train[0].shape

In [ ]:
x_train[0]

In [ ]:
y_train

<a id="ValData"></a>
+ ### Importing Validation Datasets

In [ ]:
# Import validation dataset...
x_val, y_val = load_data(files_dir='/val')

print(x_val.shape)
print(y_val.shape)

In [ ]:
y_val

<a id="TestData"></a>
+ ### Importing Test Datasets

In [ ]:
# Import test dataset...
x_test, y_test = load_data(files_dir='/test')

print(x_test.shape)
print(y_test.shape)

In [ ]:
# Counts for both classes
count_result = pd.Series(y_test).value_counts()
print('Total of Test Data : ', len(y_test), '  (0 : Normal; 1 : Pneumonia)')
print('------------------')
print(count_result)
print('------------------')
print('1 :  ', count_result[1]/sum(count_result))
print('0 :  ', count_result[0]/sum(count_result))

In [ ]:
y_test[:10]

<a id='CNNModel'></a>
## 3. CNN Model by *tf.keras*

<a id="Forwardpropagation"></a>
+ ### Forward Propagation - with Batch Normalization and Dropout
    + Conv2D layer
    + 1x1 Convolution ([Ref]: Prof Andrew Ng, "Inception Module", https://www.youtube.com/watch?v=KfV8CJh7hE0)

In [ ]:
model = Sequential([
    Conv2D(32, (5,5), activation='relu', padding='same', 
           input_shape=(224,224,3), name='Conv1_1'),
    BatchNormalization(name='bn1_1'),
    Conv2D(32, (5,5), activation='relu', padding='same', name='Conv1_2'),
    BatchNormalization(name='bn1_2'),
    Conv2D(32, (5,5), activation='relu', padding='same', name='Conv1_3'),
    BatchNormalization(name='bn1_3'),
    MaxPooling2D((2,2), name='MaxPool1'),
    Dropout(0.25),
    
    Conv2D(48, (3,3), activation='relu', padding='same', name='Conv2_1'),
    BatchNormalization(name='bn2_1'),
    Conv2D(48, (3,3), activation='relu', padding='same', name='Conv2_2'),
    BatchNormalization(name='bn2_2'),
    Conv2D(48, (3,3), activation='relu', padding='same', name='Conv2_3'),
    BatchNormalization(name='bn2_3'),    
    MaxPooling2D((2,2), name='MaxPool2'),
    Dropout(0.25),

    Conv2D(64, (3,3), activation='relu', padding='same', name='Conv3_1'),
    BatchNormalization(name='bn3_1'),
    Conv2D(64, (3,3), activation='relu', padding='same', name='Conv3_2'),
    BatchNormalization(name='bn3_2'),
    Conv2D(64, (3,3), activation='relu', padding='same', name='Conv3_3'),
    BatchNormalization(name='bn3_3'),
    MaxPooling2D((2,2), name='MaxPool3'),
    Dropout(0.25),
    
    # ----------------------------------------------------------------------
    # Using "1x1 convolution layer" to lower the complexity of computing
    # [Ref]: Prof Andrew Ng, "Inception Module", 
    #        https://www.youtube.com/watch?v=KfV8CJh7hE0
    # ----------------------------------------------------------------------
    Conv2D(16, (1,1), activation='relu', padding='same', name='Conv4_1_1x1'),
    BatchNormalization(name='bn4_1_1x1'),
    Conv2D(64, (3,3), activation='relu', padding='same', name='Conv4_2'),
    BatchNormalization(name='bn4_2'),
    Conv2D(32, (1,1), activation='relu', padding='same', name='Conv4_3_1x1'),
    BatchNormalization(name='bn4_3_1x1'),
    Conv2D(128, (3,3), activation='relu', padding='same', name='Conv4_4'),
    BatchNormalization(name='bn4_4'),
    MaxPooling2D((2,2), name='MaxPool4'),
    Dropout(0.25),

    # Using "1x1 convolution layer" 
    Conv2D(32, (1,1), activation='relu', padding='same', name='Conv5_1_1x1'),
    BatchNormalization(name='bn5_1_1x1'),
    Conv2D(128, (3,3), activation='relu', padding='same', name='Conv5_2'),
    BatchNormalization(name='bn5_2'),
    Conv2D(64, (1,1), activation='relu', padding='same', name='Conv5_3_1x1'),
    BatchNormalization(name='bn5_3_1x1'),
    Conv2D(256, (3,3), activation='relu', padding='same', name='Conv5_4'),
    BatchNormalization(name='bn5_4'),
    MaxPooling2D((2,2), name='MaxPool5'),
    Dropout(0.25),
    
    # Using "1x1 convolution layer" 
    Conv2D(64, (1,1), activation='relu', padding='same', name='Conv6_1x1'),
    BatchNormalization(name='bn6_1x1'),
    Conv2D(512, (3,3), activation='relu', padding='same', name='Conv6_2'),
    BatchNormalization(name='bn6_2'),
    
    Conv2D(128, (1,1), activation='relu', padding='same', name='Conv7_1x1'),
    BatchNormalization(name='bn7_1x1'),
    Conv2D(1024, (3,3), activation='relu', name='Conv7_2'),
    BatchNormalization(name='bn7_3'),
    GlobalAveragePooling2D(name='GlobalAveragePool_1'),
    Dropout(0.5),
    
    #Flatten(),
    #Dense(64, activation='relu', name='fc'), 
    #BatchNormalization(name='bn_fc'),
    #Dropout(0.5),
    Dense(1, name='Output')   #  activation='sigmoid' =>  BinaryCrossentropy(logits=True)
])

<a id="ModelSummary"></a>
+ ### Model Summary & Plotting the Model

In [ ]:
model.summary()

> + ### 此處可見 CNN Model 有 2,669,361 個參數需要進行訓練、調校！


In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=85)

<a id="StartTraining"></a>
## 4. Start Training with Data Augmentation

<a id="SettingHyperparameters"></a>
+ ### Setting Hyperparameters for Training Process
>+ 由於 validation dataset 只有 16 筆影像資料 (太少了些)，因此，直接將 training datasets (5216 images) 分割出 4200 筆的 training 資料 (80.5% 資料量)，其餘的 1016 張 X-ray 影像資料做為 validation 資料集。
>+ **[ Learning-Rate Decay ] : basic learning rate = 0.001**
    + 在 STAGE 1 中，設定 basic learning rate 的 1/10 為 decay rate，取 epoch 數為 10 時，每個 epoch 的 GPU 運算時間約為 20 秒。
    + 在 STAGE 2 中，設定 learning_rate 為 basic learning rate 的 1/10 (decay rate =  learning_rate/100)，取 epoch 數為 20，每個 epoch 的 GPU 運算時間約為 45 秒。

In [ ]:
batch_size = 16
epochs_stage_1 = 10
epochs_stage_2 = 20
train_data_num = 4200

<a id="Stage1"></a>
+ ### STAGE 1 : Coarse Training *without* Data Augmentation

<a id="Backpropagation1"></a>
> + ### Backpropagation - *Optimizer*, *Loss Function* & *Accuracy* for < STAGE 1 >

In [ ]:
# Adam Optimizer with Learning-rate Decay 
basic_learning_rate = 0.001
opt = Adam(lr=basic_learning_rate, decay=basic_learning_rate/10.)

model.compile(optimizer=opt,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
## data_augmentation = False
print('Not using data augmentation.')
epochs = epochs_stage_1
history_no_data_aug = model.fit(x_train[:train_data_num], y_train[:train_data_num],
                               batch_size=batch_size,
                               epochs=epochs,
                               validation_data=(x_train[train_data_num:], y_train[train_data_num:]),
                               # validation_data=(x_val, y_val),
                               shuffle=False)

In [ ]:
history_no_data_aug.history.keys()

> ### Validation-Curve Diagrams for STAGE 1 - Training without Data Augmentation

In [ ]:
acc = history_no_data_aug.history['accuracy']
val_acc = history_no_data_aug.history['val_accuracy']

loss = history_no_data_aug.history['loss']
val_loss = history_no_data_aug.history['val_loss']

epochs_range = range(1, epochs + 1)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylim(0, 1)
plt.xticks(epochs_range)
plt.title('Training and Validation Accuracy - without Data Augmentation')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylim(0, 1)
plt.xticks(epochs_range)
plt.title('Training and Validation Loss - without Data Augmentation')
plt.show()

> ### Evaluation for Test Datasets in < STAGE 1 > 

In [ ]:
# Score trained model.
loss, acc = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', acc)

In [ ]:
# Get predictions
preds = model.predict(x_test)

In [ ]:
preds.shape

In [ ]:
y_pred = []
for i in range(len(preds)):
    if preds[i] > 0.5 : 
        y_pred.append(1) 
    else: 
        y_pred.append(0)
        
print(' y_pred = ', np.array(y_pred[:10]))
print(' y_test = ', y_test[:10])

> ### Confusion Matrix for < STAGE 1 >
>                        ---------------------------------
>                        |               |               |
>                        |     true      |     false     |
>           Normal : 0   |   negative    |    positive   |
>                        |     (tn)      |      (fp)     |
>      true              |               |               |
>      value             ---------------------------------
>                        |               |               |
>                        |    false      |     true      |
>         Pneumonia : 1  |   negative    |    positive   |
>                        |     (fn)      |      (tp)     |
>                        |               |               |
>                        ---------------------------------
>                            Normal : 0     Pneumonia : 1
>
>                                 predicted value

In [ ]:
mat = confusion_matrix(y_test, y_pred)
print(mat)

plt.figure(figsize=(8,6))
sns.heatmap(mat, square=False, annot=True, fmt ='d', cbar=True, annot_kws={"size": 16})
plt.title('0 : Normal   1 : Pneumonia', fontsize = 20)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.xlabel('predicted value', fontsize = 20)
plt.ylabel('true value', fontsize = 20)
plt.show()

> ### Calculating *precision*, *recall*, *accuracy*, *F1_score* & *F2_score* for < STAGE 1 >

In [ ]:
# Calculate Precision and Recall
tn, fp, fn, tp = mat.ravel()
print('tn = {}, fp = {}, fn = {}, tp = {} '.format(tn, fp, fn, tp))

precision = tp/(tp+fp)
recall = tp/(tp+fn)
accuracy = (tp+tn)/(tp+tn+fp+fn)
f1_score = 2. * precision * recall / (precision + recall)
f2_score = 5. * precision * recall / (4. * precision + recall)

print("\nTest Recall of the model \t = {:.4f}".format(recall))
print("Test Precision of the model \t = {:.4f}".format(precision))
print("Test Accuracy of the model \t = {:.4f}".format(accuracy))
print("\nTest F1 score of the model \t = {:.4f}".format(f1_score))
print("\nTest F2 score of the model \t = {:.4f}".format(f2_score))

<a id="Stage2"></a>
+ ### STAGE 2 : Fine Training *with* Data Augmentation

<a id="Backpropagation2"></a>
> + ### Backpropagation - *Optimizer*, *Loss Function* & *Accuracy* for < STAGE 2 >

In [ ]:
# Adam Optimizer with Learning-rate Decay 
lr_with_decay = basic_learning_rate / 10.
opt = Adam(lr=lr_with_decay, decay=lr_with_decay/100.)

model.compile(optimizer=opt,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

> ### Data Augmentation Function

In [ ]:
def data_augm():
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.05,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.05,
        # rotation_range=20,
        horizontal_flip=True,  # Randomly flip inputs horizontally.
        # vertical_flip=True,  # Randomly flip inputs vertically.
        # zoom_range=[0.95, 1.05] # Range for random zoom
    )
    return datagen

> ### Training with Data Augmentation

In [ ]:
print('With data augmentation.')
datagen = data_augm()
epochs = epochs_stage_2

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train[:train_data_num])

# Fit the model on the batches generated by datagen.flow().
history_data_aug = model.fit_generator(datagen.flow(x_train[:train_data_num], y_train[:train_data_num], 
                                                    batch_size=batch_size),
                                                    epochs=epochs,
                                                    validation_data=(x_train[train_data_num:], y_train[train_data_num:]),
                                                    # validation_data=(x_val, y_val),
                                                    workers=4)

> ### Validation-Curve Diagrams for STAGE 2 - Training *with* Data Augmentation

In [ ]:
acc = history_data_aug.history['accuracy']
val_acc = history_data_aug.history['val_accuracy']

loss = history_data_aug.history['loss']
val_loss = history_data_aug.history['val_loss']

epochs_range = range(1, epochs + 1)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylim(0, 1)
plt.xticks(epochs_range)
plt.title('Training and Validation Accuracy with Data Augmentation')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylim(0, 1)
plt.xticks(epochs_range)
plt.title('Training and Validation Loss with Data Augmentation')
plt.show()

> ### Evaluation with Test Dataset for < STAGE 2 >

In [ ]:
# Score trained model.
loss, acc = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', acc)

In [ ]:
# Get predictions
preds = model.predict(x_test)

In [ ]:
preds.shape

In [ ]:
y_pred = []
for i in range(len(preds)):
    if preds[i] > 0.5 : 
        y_pred.append(1) 
    else: 
        y_pred.append(0)
        
print(' y_pred = ', np.array(y_pred[:10]))
print(' y_test = ', y_test[:10])

> ### Confusion Matrix for < STAGE 2 >
>                        ---------------------------------
>                        |               |               |
>                        |     true      |     false     |
>           Normal : 0   |   negative    |    positive   |
>                        |     (tn)      |      (fp)     |
>      true              |               |               |
>      value             ---------------------------------
>                        |               |               |
>                        |    false      |     true      |
>         Pneumonia : 1  |   negative    |    positive   |
>                        |     (fn)      |      (tp)     |
>                        |               |               |
>                        ---------------------------------
>                            Normal : 0     Pneumonia : 1
>
>                                 predicted value

In [ ]:
mat = confusion_matrix(y_test, y_pred)
print(mat)

plt.figure(figsize=(8,6))
sns.heatmap(mat, square=False, annot=True, fmt ='d', cbar=True, annot_kws={"size": 16})
plt.title('0 : Normal   1 : Pneumonia', fontsize = 20)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.xlabel('predicted value', fontsize = 20)
plt.ylabel('true value', fontsize = 20)
plt.show()

### Calculating *precision*, *recall*, *accuracy*, *F1_score* & *F2_score* for < STAGE 2 >

In [ ]:
# Calculate Precision and Recall
tn, fp, fn, tp = mat.ravel()
print('tn = {}, fp = {}, fn = {}, tp = {} '.format(tn, fp, fn, tp))

precision = tp/(tp+fp)
recall = tp/(tp+fn)
accuracy = (tp+tn)/(tp+tn+fp+fn)
f1_score = 2. * precision * recall / (precision + recall)
f2_score = 5. * precision * recall / (4. * precision + recall)

print("\nTest Recall of the model \t = {:.4f}".format(recall))
print("Test Precision of the model \t = {:.4f}".format(precision))
print("Test Accuracy of the model \t = {:.4f}".format(accuracy))
print("\nTest F1 score of the model \t = {:.4f}".format(f1_score))
print("\nTest F2 score of the model \t = {:.4f}".format(f2_score))

<a id="Result"></a>
## 5. Results
> + 由以上結果可知：經過 30 epochs (i.e., Stage 1 + Stage 2) 的訓練，**Recall** 值接近 100%；這代表預測模型的 false-negative 誤判部分 (亦即，將 Pneumonia 誤判成 Normal 的狀況) 將會下降趨近 0。
> + 然而， **F1 score** 仍然還有改進的空間；這是因為 false-positive 誤診部分 (亦即，將 Normal 誤判成 Pneumonia 的狀況) 造成 **Precision** 的精確值偏低的緣故。
> + 請想想看，如何改進模型或調校預測模型參數 (hyperparameters)，進而能夠使其 **F1 score** 數值上升。  Good luck！

以下將 From-Coarse-to-Fine 全程訓練過程的 Learning Curves 繪製如下，做為參考：

In [ ]:
acc_total = history_no_data_aug.history['accuracy'] + history_data_aug.history['accuracy']
val_acc_total = history_no_data_aug.history['val_accuracy'] + history_data_aug.history['val_accuracy']

loss_total = history_no_data_aug.history['loss'] + history_data_aug.history['loss']
val_loss_total = history_no_data_aug.history['val_loss'] + history_data_aug.history['val_loss']

In [ ]:
initial_epochs = epochs_stage_1
total_epochs = epochs_stage_1 + epochs_stage_2
epochs_range = range(1, total_epochs + 1)

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(epochs_range, acc_total, label='Training Accuracy')
plt.plot(epochs_range, val_acc_total, label='Validation Accuracy')
plt.ylim([0, 1])
plt.xticks(range(1,total_epochs+1,1))
plt.plot([initial_epochs,initial_epochs],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(epochs_range, loss_total, label='Training Loss')
plt.plot(epochs_range, val_loss_total, label='Validation Loss')
plt.ylim([0, 1])
plt.xticks(range(1,total_epochs+1,1))
plt.plot([initial_epochs,initial_epochs],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

<a id="SavingEntireModel"></a>
## 6. Saving the Entire Model with HDF5 Format

In [ ]:
# Saving the entire model to a HDF5 file：
# The '.h5' extension is for the HDF5 format.
model.save('PD_HDF5_model.h5')

In [ ]:
# Reloading the HDF5 model, including its weights and the optimizer.
HDF5_model = tf.keras.models.load_model('PD_HDF5_model.h5')

# Show the model architecture
HDF5_model.summary()

In [ ]:
# Evaluate the restored HDF5 model
loss, acc = HDF5_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', acc)

In [ ]:
# submission = pd.concat([pd.Series(range(1,(len(pred)+1)),name = "ImageId"),preds],axis = 1)
data_subm = {'ImageId': pd.Series(range(1,(len(y_pred)+1))), 'Prediction': y_pred}
submission = pd.DataFrame(data_subm)
submission = submission.applymap(str)

submission.to_csv("submission.csv",index=False)